In [ ]:
#Most Recently Analyzed 
import numpy as np
import cv2

cap = cv2.VideoCapture('/Users/sofielysenko/Desktop/test2.mp4')

import csv
Image, Emotion, Point1, Point2, Point3, Point4, Point5, Point6 = 'Image', 'Emotion', 'Point1', 'Point2', 'Point3', 'Point4', 'Point5', 'Point6'
csvRow = [Image, Emotion, Point1, Point2, Point3, Point4, Point5, Point6]
csvfile = "FS2.csv"


myFile = open('FS1.csv', 'w')  
with myFile:  
    myFields = ['Image','Emotion', 'Point1', 'Point2', 'Point3', 'Point4', 'Point5', 'Point6']
    writer = csv.DictWriter(myFile, fieldnames=myFields)    
    writer.writeheader()


# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.1,
                       minDistance = 7,
                       blockSize = 10 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
i=0
while(1):
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    print(p1)
    with open(csvfile, "a") as fp:
        wr = csv.writer(fp, dialect='excel')
        wr.writerow(["frame" + str(i), 'Negative', p1[1], p1[2], p1[3], p1[4], p1[5], p1[6]])
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

In [ ]:
import cv2
import numpy as np
cap = cv2.VideoCapture("/Users/sofielysenko/Desktop/motion/testmotion.mp4")
ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255

import csv
Image, Emotion, eye_bottom = 'Image', 'Emotion', 'mag'
csvRow = [Image, Emotion, eye_bottom]
csvfile = "OFR.csv"


myFile = open('OFR.csv', 'w')  
with myFile:  
    myFields = ['Image','Emotion', 'mag']
    writer = csv.DictWriter(myFile, fieldnames=myFields)    
    writer.writeheader()
   
while(1):
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame2',bgr)
    print(mag)
    print(ang)
    import csv
    Image, eye_bottom, Emotion = 'Image','Emotion', 'mag'
    csvRow = [Image, Emotion, eye_bottom]
    csvfile = "OFR.csv"
   
    with open(csvfile, "a") as fp:
        wr = csv.writer(fp, dialect='excel')
        wr.writerow([frame2, 'Negative', mag])
        wr.writerow([frame2, 'Negative', ang])
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv2.imwrite('opticalfb.png',frame2)
        cv2.imwrite('opticalhsv.png',bgr)
    prvs = next
cap.release()
cv2.destroyAllWindows()

In [ ]:
p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

In [ ]:
#Calculate Centroid
import numpy as np
import cv2 as cv
img = cv.imread('/Users/sofielysenko/Desktop/positiveframes/7.jpg',0)
ret,thresh = cv.threshold(img,127,255,0)
im2,contours,hierarchy = cv.findContours(thresh, 1, 2)
cnt = contours[0]
M = cv.moments(cnt)
print(M)

cx = int(M['m10']/M['m00'])
cy = int(M['m01']/M['m00'])